In [56]:
import os

## Get Reference Article Text

In [57]:
FOLDER = "output-final-set-citing-accuracy"
OUTPUT_FOLDER = "refefence"

In [61]:
!mkdir {OUTPUT_FOLDER}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [62]:
for file in os.listdir(FOLDER):
    if "PMC" not in file:
        continue
    for cit in os.listdir(f"{FOLDER}/{file}"):
        if cit.endswith('ann'):
            continue
        with open(f"{FOLDER}/{file}/{cit}") as f:
            content = f.read()
            spl = content.split("**************************************************************")

        
        with open(f"{OUTPUT_FOLDER}/{file}.txt", 'w') as f:
            f.write(spl[1].strip())
        break

## Get Contexts

In [66]:
FOLDER = "output-final-set-citing-accuracy"

In [67]:
import spacy
import spacy_transformers
nlp = spacy.load("en_core_sci_md")

In [68]:
from tqdm.notebook import tqdm

In [71]:
def overlap(start, end, s, e):
    return start <= e and s <= end

res = {}
files = os.listdir(FOLDER)
for i in tqdm(range(len(files))):
    file = files[i]
    if "PMC" not in file:
        continue
    print(file)
    res[file] = {}
    for cit in os.listdir(f"{FOLDER}/{file}"):
        if cit.endswith('ann'):
            citing_article = cit[:-4]
            with open(f"{FOLDER}/{file}/{citing_article}.txt") as f:
                text = f.read()
                citing_text = text.split("**************************************************************")[0]
                doc = nlp(citing_text)
                sentences = [str(s) for s in list(doc.sents) if str(s) != '\n\n']
                
            with open(f"{FOLDER}/{file}/{citing_article}.ann") as f:
                lines = f.readlines()
                if lines[0].split("\t")[1].split(" ")[0] not in ["Citation", "MultiCitation"]:
                    print("ERROR", file, cit)
                s, e = lines[0].split("\t")[1].split(" ")[1], lines[0].split("\t")[1].split(" ")[2]
                s, e = int(s), int(e)
                
                for sentence in sentences:
                    a, b = citing_text.find(sentence), citing_text.find(sentence)+len(sentence)
                    if overlap(s, e, a, b):
                        res[file][citing_article] = sentence

                        

        
        

  0%|          | 0/12 [00:00<?, ?it/s]

010_PMC3650111
013_PMC7199903
019_PMC7581548
020_PMC7787219
012_PMC7164881
021_PMC8078878
016_PMC7299280
011_PMC7164482
ERROR 011_PMC7164482 PMC8415352_2.ann
018_PMC7578095
017_PMC7418704
015_PMC7262788
014_PMC7227586


In [74]:
import json
with open("contexts.json", 'w') as f:
    json.dump(res, f)
    

## Split Sentences

In [77]:
OUTPUT_FOLDER = "refefence"

In [81]:
all_files = os.listdir(OUTPUT_FOLDER)

In [82]:
for i, file in enumerate(all_files):
    if file.endswith('txt'):
        print(i, file)
        with open(f"{OUTPUT_FOLDER}/{file}", encoding='utf-8') as f:
            text = f.read()
            doc = nlp(text)
            sentences_all = [str(s) for s in list(doc.sents) if str(s) != '\n\n']
            sentences = []
            for s in sentences_all:
                if "references" in s.lower():
                    break
                sentences.append(s)
            sentences = [s.strip() for s in sentences if s.strip().count(' ') >= 2]
        
        if not os.path.isdir("Sentences"):
            os.mkdir("Sentences")
        
        with open(f'Sentences/{file[:-4]}.json', 'w') as g:
            json.dump(sentences, g)

0 010_PMC3650111.txt
1 011_PMC7164482.txt
2 015_PMC7262788.txt
3 019_PMC7581548.txt
4 013_PMC7199903.txt
5 017_PMC7418704.txt
6 020_PMC7787219.txt
7 012_PMC7164881.txt
8 016_PMC7299280.txt
9 021_PMC8078878.txt
11 014_PMC7227586.txt
12 018_PMC7578095.txt
